In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import json
import tensorflow as tf
from numpy import *
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
from tensorflow.keras import layers,datasets
from sklearn.preprocessing import LabelEncoder,MinMaxScaler
from tensorflow import keras
import gc
import warnings
warnings.filterwarnings('ignore')
import lightgbm as lgb
import category_encoders as ce
from sklearn.metrics import f1_score,classification_report,roc_auc_score

In [105]:
from impala.dbapi import connect
from impala.util import as_pandas

In [ ]:
# 连接数据库
conn_hive = connect(host='192.168.0.88', port=10000, auth_mechanism='PLAIN', user='hadoop',
                    password='Ycjh8FxiaoMtShZRd3-97%3hCEL0CK4ns1w', database='default')
cursor_hive = conn_hive.cursor()
conn_mysql = pymysql.connect(host='192.168.0.79', port=3306, user='readonly',
                             passwd='Kc&r+z1ql9x8za4kzfk761weq8ozGv3ZpI;RMe,#+s%c>t', db='db_stdata')

In [111]:
# 获取score_image (分数大于550的image_id)
def get_score_image():
    conn = connect(host='192.168.0.88', port=10000, auth_mechanism='PLAIN', user='hadoop',
                    password='Ycjh8FxiaoMtShZRd3-97%3hCEL0CK4ns1w', database='default')
    cursor = conn.cursor()
    sql_engine = 'set hive.execution.engine=tez'
    sql = 'select image_id from dws.dws_image_score_d where media_id=10 and score>=550 and dt>=date_sub(CURRENT_DATE,1) group by image_id'
    cursor.execute(sql_engine)
    cursor.execute(sql)
    result = as_pandas(cursor)
    # 关闭链接
    cursor.close()
    conn.close()
    
    return result

In [112]:
scroe_image = get_score_image()

In [ ]:
from impala.dbapi import connect
from impala.util import as_pandas

conn_hive = connect(host='192.168.0.88', port=10000, auth_mechanism='PLAIN', user='hadoop',
                    password='Ycjh8FxiaoMtShZRd3-97%3hCEL0CK4ns1w', database='default')
cursor_hive = conn_hive.cursor()
cursor.execute('show databases')
result_df = as_pandas(cursor)

In [3]:
pd.set_option('display.max_columns',None)

In [4]:
scroe_image = np.array([25547,
25562,
25825,
25907,
25919,
26241,
26441,
27672,
28397,
28541,
28590,
28594,
28609,
28678,
28714,
28722,
28738,
28744,
28800,
28803,
28826,
28849,
28891,
28976,
29000,
29045,
29075,
29147,
29160,
29161,
29167,
29226,
29274,
29332,
29333,
29356,
29400,
29409,
29433,
29436,
29437,
29451,
29466,
29477,
29512])

In [119]:
plan_info = pd.read_csv('./now_plan.csv')
now_plan_roi = pd.read_csv('./now_plan_roi.csv')

In [114]:
plan_info.head()

,ad_account_id,ad_id,ad_group_id,ad_name,media_id,plan_id,game_id,channel_id,source_id,platform,launch_op_id,amount,paper_amount,click_num,pv,active_num,create_role_num,new_user_num,bid_mode,bid_status,bid,bid2,budget_mode,budget,ad_start_time,ad_end_time,ad_info,opt_status,status,syn_time,remark,update_time,create_time,origin,ad_resource_id,image_id,inventory_type
0,6802,1690567592308765,1690565674896412,yo-末世-2.2_18,10,450159,1001049,20754,460024,NaN,12447,NaN,NaN,NaN,NaN,NaN,NaN,NaN,OCPM,3,0.0,2666.0,1,2888,NaN,NaN,"{""app_type"":""APP_ANDROID"",""smart_bid_type"":""SM...",1,1,2021-2-2 16:21:13,NaN,2021-2-2 16:21:12,2021-2-2 15:28:18,1.0,NaN,NaN,"INVENTORY_FEED,INVENTORY_AWEME_FEED,INVENTORY_..."
1,6802,1690567596678221,1690565674896412,yo-末世-2.2_14,10,450161,1001049,20754,460028,NaN,12447,NaN,NaN,NaN,NaN,NaN,NaN,NaN,OCPM,3,0.0,2666.0,1,2888,NaN,NaN,"{""app_type"":""APP_ANDROID"",""smart_bid_type"":""SM...",1,1,2021-2-2 17:01:26,NaN,2021-2-2 17:01:26,2021-2-2 15:28:18,1.0,NaN,NaN,"INVENTORY_FEED,INVENTORY_HOTSOON_FEED,INVENTOR..."
2,6802,1690567596776460,1690565674896412,yo-末世-2.2_12,10,450156,1001049,20754,460021,NaN,12447,NaN,NaN,NaN,NaN,NaN,NaN,NaN,OCPM,3,0.0,2666.0,1,2888,NaN,NaN,"{""app_type"":""APP_ANDROID"",""smart_bid_type"":""SM...",1,1,2021-2-2 17:01:26,NaN,2021-2-2 17:01:26,2021-2-2 15:28:18,1.0,NaN,NaN,"INVENTORY_HOTSOON_FEED,INVENTORY_UNION_SLOT,IN..."
3,6802,1690567791708244,1690565674896412,yo-末世-2.2_16,10,450182,1001049,20754,460047,NaN,12447,NaN,NaN,NaN,NaN,NaN,NaN,NaN,OCPM,3,0.0,2666.0,1,2888,NaN,NaN,"{""app_type"":""APP_ANDROID"",""smart_bid_type"":""SM...",1,1,2021-2-2 17:01:26,NaN,2021-2-2 17:01:26,2021-2-2 15:31:29,1.0,NaN,NaN,"INVENTORY_HOTSOON_FEED,INVENTORY_UNION_SLOT,IN..."
4,6802,1690567792089123,1690565674896412,yo-末世-2.2_13,10,450183,1001049,20754,460048,NaN,12447,NaN,NaN,NaN,NaN,NaN,NaN,NaN,OCPM,3,0.0,2666.0,1,2888,NaN,NaN,"{""app_type"":""APP_ANDROID"",""smart_bid_type"":""SM...",1,1,2021-2-2 16:21:13,NaN,2021-2-2 16:21:12,2021-2-2 15:31:29,1.0,NaN,NaN,"INVENTORY_AWEME_FEED,INVENTORY_FEED,"


In [115]:
now_plan_roi.head()

,channel_id,source_id,amount,create_role_num,new_role_money,pay_role_user_num
0,6770,446372,13430.05,2604,566.0,15.0
1,6770,446377,8.64,1,12.0,1.0
2,6770,446380,31.55,2,NaN,NaN
3,6770,447387,0.29,0,NaN,NaN
4,6770,447389,1728.19,31,NaN,NaN


In [120]:
plan_info.drop('inventory_type', axis=1, inplace=True)

In [121]:
plan_info.dropna(how='all', inplace=True, axis=1)
plan_info.dropna(subset=['ad_info'],inplace=True)

In [122]:
# 解析json
plan_info['ad_info'] = plan_info['ad_info'].apply(json.loads)
temp = plan_info['ad_info'].apply(pd.Series)
plan_info = pd.concat([plan_info,temp], axis=1)
plan_info.drop('ad_info', axis=1, inplace=True)
temp = plan_info['audience'].apply(pd.Series)
plan_info = pd.concat([plan_info,temp], axis=1)
plan_info.drop('audience', axis=1, inplace=True)
temp = plan_info['action'].apply(pd.Series)
plan_info = pd.concat([plan_info,temp], axis=1)
plan_info.drop('action', axis=1, inplace=True)
plan_info.dropna(how='all', inplace=True, axis=1)

In [123]:
plan_info = plan_info[['ad_account_id', 'game_id', 'channel_id', 'source_id', 'bid2',
       'create_time', 'smart_bid_type', 'hide_if_exists', 'budget',
       'delivery_range', 'adjust_cpa', 'inventory_type', 'hide_if_converted',
       'flow_control_mode', 'schedule_time', 'cpa_bid', 'auto_extend_enabled',
       'gender', 'city', 'platform', 'launch_price',
       'retargeting_tags_exclude', 'interest_categories', 'interest_words',
       'ac', 'android_osv', 'location_type', 'retargeting_tags_include',
       'retargeting_type', 'ios_osv', 'interest_action_mode', 'age',
       'action_categories', 'action_days', 'action_scene', 'action_words']]

In [124]:
plan_info['bid2'].max()

7000.0

In [12]:
plan_info['platform'] = plan_info['platform'].astype(str)
plan_info['platform'] = plan_info['platform'].map({"['ANDROID']":1,"['IOS']":2})

In [13]:
plan_info.shape

(2028, 37)

In [125]:
image_info = pd.read_csv('./image_info.csv')
df = pd.merge(plan_info,image_info,on=['channel_id','source_id'],how='left')

In [116]:
image_info.head()

,channel_id,source_id,image_id,label_ids
0,6770,254303,19395.0,NaN
1,6770,254304,19418.0,NaN
2,6770,254305,19418.0,NaN
3,6770,254306,19395.0,NaN
4,6770,254307,19511.0,NaN


In [126]:
df = pd.merge(df,now_plan_roi,on=['channel_id','source_id'],how='inner')
df = df[df['pay_role_user_num'].notna()]
df['roi'] = df['new_role_money']/df['amount']
df['pay_cost'] = df['amount']/df['pay_role_user_num']

In [16]:
df['platform'] = df['platform'].astype(int)
df['pay_cost_ok'] = df.apply(lambda x:1 if (x.platform==1)&(x.pay_cost<=5000) else (1 if 
                            (x.platform==2)&(x.pay_cost<=8000) else 0), axis=1)
df['amount_ok'] = df.apply(lambda x:1 if (x.platform==1)&(x.amount>=5000) else (1 if 
                            (x.platform==2)&(x.amount>=8000) else 0), axis=1)
df = df[(df['amount_ok']==1)&(df['pay_cost_ok']==1)]
df = df[df['roi']>0.01]

In [17]:
df.dropna(subset=['image_id'],inplace=True)
df['image_id'] = df['image_id'].astype(int)

In [ ]:
df['pay_rate'] = df['pay_role_user_num'] / df['create_role_num']

In [38]:
df.drop(['action_words','interest_words'],axis=1,inplace=True)

In [39]:
df.shape

(129, 46)

In [20]:
df.head()

,ad_account_id,game_id,channel_id,source_id,bid2,create_time,smart_bid_type,hide_if_exists,budget,budget,delivery_range,adjust_cpa,inventory_type,hide_if_converted,flow_control_mode,schedule_time,cpa_bid,auto_extend_enabled,gender,city,platform,launch_price,retargeting_tags_exclude,interest_categories,interest_words,ac,android_osv,location_type,retargeting_tags_include,retargeting_type,ios_osv,interest_action_mode,age,action_categories,action_days,action_scene,action_words,image_id,label_ids,amount,create_role_num,new_role_money,pay_role_user_num,roi,pay_cost,pay_cost_ok,amount_ok,pay_rate
2,6417,1001379,7162,455943,2566.0,2021-1-30 01:06:18,SMART_BID_CONSERVATIVE,0,3000,3000.0,DEFAULT,1,"[INVENTORY_AWEME_FEED, INVENTORY_FEED]",ADVERTISER,FLOW_CONTROL_MODE_FAST,0000000000000000000000001111111111111111111111...,2566.0,0,GENDER_MALE,[],1,"[0, 11000]",[],[],[],"[4G, WIFI]",8.0,CURRENT,[],NONE,NONE,UNLIMITED,"[AGE_BETWEEN_18_23, AGE_BETWEEN_24_30, AGE_BET...",NaN,NaN,NaN,NaN,28803,127,17448.17,565,3456.0,10.0,0.198072,1744.817000,1,1,0.017699
3,6417,1001379,7162,455940,2566.0,2021-1-30 01:06:18,SMART_BID_CONSERVATIVE,0,3000,3000.0,DEFAULT,1,"[INVENTORY_AWEME_FEED, INVENTORY_FEED]",APP,FLOW_CONTROL_MODE_FAST,0000000000000000000000001111111111111111111111...,2566.0,1,NONE,[],1,"[0, 11000]",[],[],[],"[4G, WIFI]",9.0,CURRENT,[],NONE,NONE,UNLIMITED,"[AGE_BETWEEN_24_30, AGE_BETWEEN_31_40, AGE_BET...",NaN,NaN,NaN,NaN,28803,127,20783.89,1266,622.0,9.0,0.029927,2309.321111,1,1,0.007109
5,6417,1001379,7162,455939,2566.0,2021-1-30 01:06:18,SMART_BID_CONSERVATIVE,0,3000,3000.0,DEFAULT,1,"[INVENTORY_AWEME_FEED, INVENTORY_FEED]",CUSTOMER,FLOW_CONTROL_MODE_FAST,0000000000000000000000001111111111111111111111...,2566.0,0,GENDER_MALE,[],1,"[0, 11000]",[],[],[],[],7.1,CURRENT,[],NONE,NONE,UNLIMITED,"[AGE_BETWEEN_18_23, AGE_BETWEEN_24_30, AGE_BET...",NaN,NaN,NaN,NaN,28803,127,16570.17,772,372.0,5.0,0.022450,3314.034000,1,1,0.006477
16,6804,1001379,7162,455469,2566.0,2021-1-29 21:20:47,SMART_BID_CUSTOM,0,3000,3000.0,DEFAULT,0,"[INVENTORY_AWEME_FEED, INVENTORY_FEED]",APP,FLOW_CONTROL_MODE_FAST,0000000000000000000000000000001111111111111111...,2566.0,1,GENDER_MALE,[],1,"[2000, 11000]",[],[],[],"[4G, WIFI]",8.0,CURRENT,[],NONE,NONE,UNLIMITED,"[AGE_BETWEEN_18_23, AGE_BETWEEN_24_30, AGE_BET...",NaN,NaN,NaN,NaN,28803,127,16881.93,462,1056.0,6.0,0.062552,2813.655000,1,1,0.012987
19,6804,1001379,7162,455466,2566.0,2021-1-29 21:20:47,SMART_BID_CUSTOM,0,3000,3000.0,DEFAULT,0,"[INVENTORY_AWEME_FEED, INVENTORY_FEED]",ADVERTISER,FLOW_CONTROL_MODE_FAST,0000000000000000000000000000111111111111111111...,2566.0,0,GENDER_MALE,[],1,"[0, 11000]",[],[],[],"[4G, WIFI]",8.0,CURRENT,[],NONE,NONE,UNLIMITED,"[AGE_BETWEEN_18_23, AGE_BETWEEN_24_30, AGE_BET...",NaN,NaN,NaN,NaN,28803,127,5706.39,230,132.0,3.0,0.023132,1902.130000,1,1,0.013043


In [22]:
# 选ad_account_id、image_id、game_id，每个账号+素材8条
ad_account_id_group = np.array([6620,6621,6867,6868])
image_id_group =np.intersect1d(df['image_id'].unique(),scroe_image)
plan = pd.DataFrame()
for ad_account in ad_account_id_group:
    for image in image_id_group:
#         print(image)
        temp = pd.DataFrame({'ad_account_id':[ad_account],'image_id':[image]})
        plan = plan.append(temp)
#         print(temp)
plan = pd.DataFrame(np.repeat(plan.values,8,axis=0), columns=plan.columns)
plan['game_id'] = 1001379

In [23]:
# 选platform测试期默认[ANDROID]
plan['platform']='[ANDROID]'

In [24]:
# 选android_osv
count_df = pd.DataFrame(data=df['android_osv'].value_counts()).reset_index()
count_df.columns = ['col','counts']
count_df['pct'] = count_df['counts'] / count_df['counts'].sum()
plan['android_osv']=plan['platform'].apply(lambda x:'NONE' if x=='[IOS]' else 
                                           np.random.choice(count_df['col'].values, 1, p=count_df['pct'].values)[0])

In [25]:
# 选ios_osv
count_df = pd.DataFrame(data=df['ios_osv'].value_counts()).reset_index()
count_df.columns = ['col','counts']
count_df['pct'] = count_df['counts'] / count_df['counts'].sum()
plan['ios_osv']=plan['platform'].apply(lambda x:'NONE' if x=='[ANDROID]' else 
                                           np.random.choice(count_df['col'].values, 1, p=count_df['pct'].values)[0])

In [95]:
# 选budget 
plan['budget']= plan['platform'].apply(lambda x:3300 if x=='[ANDROID]' else 4000)

In [26]:
# 选delivery_range 投放范围
count_df = pd.DataFrame(data=df['delivery_range'].value_counts()).reset_index()
count_df.columns = ['col','counts']
count_df['pct'] = count_df['counts'] / count_df['counts'].sum()
for i in range(plan.shape[0]):
    plan.loc[i,'delivery_range'] = np.random.choice(count_df['col'].values, 1, p=count_df['pct'].values)[0]

In [187]:
count_df = pd.DataFrame(data=df['inventory_type'].value_counts()).reset_index()

In [188]:
count_df.columns = ['col','counts']

In [185]:
for i in range(count_df.shape[0]):
    values = count_df.loc[i,'col']
    
    for value in values:
        if value== None:
            values.remove(value)
    a = np.empty(1, dtype=object)
    a[0] = values
    count_df.loc[i,'col'] = a
        

In [27]:
# 选inventory_type 创意投放位置
count_df = pd.DataFrame(data=df['inventory_type'].value_counts()).reset_index()
count_df.columns = ['col','counts']

for i in range(count_df.shape[0]):
    values = count_df.loc[i,'col']
    
    for value in values:
        if value== None:
            values.remove(value)
    a = np.empty(1, dtype=object)
    a[0] = values
    count_df.loc[i,'col'] = a
count_df['pct'] = count_df['counts'] / count_df['counts'].sum()
plan['inventory_type']=plan.apply(lambda x:np.random.choice(count_df['col'].values, 1, p=count_df['pct'].values)[0], axis=1)

In [28]:
# 选city 创意投放位置
count_df = pd.DataFrame(data=df['city'].value_counts()).reset_index()
count_df.columns = ['col','counts']
count_df['pct'] = count_df['counts'] / count_df['counts'].sum()
plan['city']=plan.apply(lambda x:np.random.choice(count_df['col'].values, 1, p=count_df['pct'].values)[0], axis=1)

In [29]:
# 选location_type 创意投放位置
count_df = pd.DataFrame(data=df['location_type'].value_counts()).reset_index()
count_df.columns = ['col','counts']
count_df['pct'] = count_df['counts'] / count_df['counts'].sum()
plan['location_type']=plan.apply(lambda x:np.random.choice(count_df['col'].values, 1, p=count_df['pct'].values)[0], axis=1)

In [30]:
# 选gender 创意投放位置
count_df = pd.DataFrame(data=df['gender'].value_counts()).reset_index()
count_df.columns = ['col','counts']
count_df['pct'] = count_df['counts'] / count_df['counts'].sum()
plan['gender']=plan.apply(lambda x:np.random.choice(count_df['col'].values, 1, p=count_df['pct'].values)[0], axis=1)

In [31]:
# 选age 创意投放位置
count_df = pd.DataFrame(data=df['age'].value_counts()).reset_index()
count_df.columns = ['col','counts']
count_df['pct'] = count_df['counts'] / count_df['counts'].sum()
plan['age']=plan.apply(lambda x:np.random.choice(count_df['col'].values, 1, p=count_df['pct'].values)[0], axis=1)

In [32]:
# 选'retargeting_type','retargeting_tags_include','retargeting_tags_exclude'  人群包定向
sample_df = df[['retargeting_type','retargeting_tags_include','retargeting_tags_exclude','pay_rate']]
sample_df = sample_df.sample(n=plan.shape[0], replace=True, weights=sample_df['pay_rate']).drop('pay_rate',axis=1).reset_index(drop=True)
plan = pd.concat([plan, sample_df], axis=1)

In [41]:
# 选'interest_action_mode','action_scene','action_days','action_categories' ,'interest_categories' 行为兴趣
sample_df = df[['interest_action_mode','action_scene','action_days','action_categories','interest_categories','pay_rate']]
sample_df = sample_df.sample(n=plan.shape[0], replace=True, weights=sample_df['pay_rate']).drop('pay_rate',axis=1).reset_index(drop=True)
plan = pd.concat([plan, sample_df], axis=1)

In [64]:
# 选'ac' 网络类型
count_df = pd.DataFrame(data=df['ac'].value_counts()).reset_index()
count_df.columns = ['col','counts']
count_df['pct'] = count_df['counts'] / count_df['counts'].sum()
plan['ac']=plan.apply(lambda x:np.random.choice(count_df['col'].values, 1, p=count_df['pct'].values)[0], axis=1)

In [67]:
# 选'launch_price' 网络类型
count_df = pd.DataFrame(data=df['launch_price'].value_counts()).reset_index()
count_df.columns = ['col','counts']
count_df['pct'] = count_df['counts'] / count_df['counts'].sum()
plan['launch_price']=plan.apply(lambda x:np.random.choice(count_df['col'].values, 1, p=count_df['pct'].values)[0], axis=1)

In [71]:
# 选'auto_extend_enabled' 智能放量
count_df = pd.DataFrame(data=df['auto_extend_enabled'].value_counts()).reset_index()
count_df.columns = ['col','counts']
count_df['pct'] = count_df['counts'] / count_df['counts'].sum()
plan['auto_extend_enabled']=plan.apply(lambda x:np.random.choice(count_df['col'].values, 1, p=count_df['pct'].values)[0], axis=1)

In [75]:
# 选'hide_if_exists' 
count_df = pd.DataFrame(data=df['hide_if_exists'].value_counts()).reset_index()
count_df.columns = ['col','counts']
count_df['pct'] = count_df['counts'] / count_df['counts'].sum()
plan['hide_if_exists']=plan.apply(lambda x:np.random.choice(count_df['col'].values, 1, p=count_df['pct'].values)[0], axis=1)

In [96]:
# 选'hide_if_converted' 
count_df = pd.DataFrame(data=df['hide_if_converted'].value_counts()).reset_index()
count_df.columns = ['col','counts']
count_df['pct'] = count_df['counts'] / count_df['counts'].sum()
plan['hide_if_converted']=plan.apply(lambda x:np.random.choice(count_df['col'].values, 1, p=count_df['pct'].values)[0], axis=1)

In [85]:
# 选'smart_bid_type','adjust_cpa','cpa_bid','bid'出价方式
sample_df = df[['smart_bid_type','adjust_cpa','cpa_bid','bid2','pay_rate']]
sample_df = sample_df[sample_df['bid2']>=1500]
sample_df = sample_df.sample(n=plan.shape[0], replace=True, weights=sample_df['pay_rate']).drop('pay_rate',axis=1).reset_index(drop=True)
plan = pd.concat([plan, sample_df], axis=1)

In [90]:
# 选'schedule_time' 
count_df = pd.DataFrame(data=df['schedule_time'].value_counts()).reset_index()
count_df.columns = ['col','counts']
count_df['pct'] = count_df['counts'] / count_df['counts'].sum()
plan['schedule_time']=plan.apply(lambda x:np.random.choice(count_df['col'].values, 1, p=count_df['pct'].values)[0], axis=1)

In [99]:
# 选'flow_control_mode' 竞价策略(投放方式)
count_df = pd.DataFrame(data=df['flow_control_mode'].value_counts()).reset_index()
count_df.columns = ['col','counts']
count_df['pct'] = count_df['counts'] / count_df['counts'].sum()
plan['flow_control_mode']=plan.apply(lambda x:np.random.choice(count_df['col'].values, 1, p=count_df['pct'].values)[0], axis=1)

In [129]:
plan.columns

Index(['ad_account_id', 'image_id', 'game_id', 'platform', 'android_osv',
       'ios_osv', 'delivery_range', 'inventory_type', 'city', 'location_type',
       'gender', 'age', 'retargeting_type', 'retargeting_tags_include',
       'retargeting_tags_exclude', 'interest_action_mode', 'action_scene',
       'action_days', 'action_categories', 'interest_categories', 'ac',
       'launch_price', 'auto_extend_enabled', 'hide_if_exists',
       'smart_bid_type', 'smart_bid_type', 'adjust_cpa', 'cpa_bid', 'bid2',
       'schedule_time', 'budget', 'hide_if_converted', 'flow_control_mode'],
      dtype='object')

In [128]:
plan.shape

(736, 33)

In [82]:
df[df['smart_bid_type']=='SMART_BID_CONSERVATIVE']

,ad_account_id,game_id,channel_id,source_id,bid2,create_time,smart_bid_type,hide_if_exists,budget,budget,delivery_range,adjust_cpa,inventory_type,hide_if_converted,flow_control_mode,schedule_time,cpa_bid,auto_extend_enabled,gender,city,platform,launch_price,retargeting_tags_exclude,interest_categories,ac,android_osv,location_type,retargeting_tags_include,retargeting_type,ios_osv,interest_action_mode,age,action_categories,action_days,action_scene,image_id,label_ids,amount,create_role_num,new_role_money,pay_role_user_num,roi,pay_cost,pay_cost_ok,amount_ok,pay_rate
2,6417,1001379,7162,455943,2566.0,2021-1-30 01:06:18,SMART_BID_CONSERVATIVE,0,3000,3000.0,DEFAULT,1,"[INVENTORY_AWEME_FEED, INVENTORY_FEED]",ADVERTISER,FLOW_CONTROL_MODE_FAST,0000000000000000000000001111111111111111111111...,2566.0,0,GENDER_MALE,[],1,"[0, 11000]",[],[],"[4G, WIFI]",8.0,CURRENT,[],NONE,NONE,UNLIMITED,"[AGE_BETWEEN_18_23, AGE_BETWEEN_24_30, AGE_BET...",NaN,NaN,NaN,28803,127,17448.17,565,3456.0,10.0,0.198072,1744.817000,1,1,0.017699
3,6417,1001379,7162,455940,2566.0,2021-1-30 01:06:18,SMART_BID_CONSERVATIVE,0,3000,3000.0,DEFAULT,1,"[INVENTORY_AWEME_FEED, INVENTORY_FEED]",APP,FLOW_CONTROL_MODE_FAST,0000000000000000000000001111111111111111111111...,2566.0,1,NONE,[],1,"[0, 11000]",[],[],"[4G, WIFI]",9.0,CURRENT,[],NONE,NONE,UNLIMITED,"[AGE_BETWEEN_24_30, AGE_BETWEEN_31_40, AGE_BET...",NaN,NaN,NaN,28803,127,20783.89,1266,622.0,9.0,0.029927,2309.321111,1,1,0.007109
5,6417,1001379,7162,455939,2566.0,2021-1-30 01:06:18,SMART_BID_CONSERVATIVE,0,3000,3000.0,DEFAULT,1,"[INVENTORY_AWEME_FEED, INVENTORY_FEED]",CUSTOMER,FLOW_CONTROL_MODE_FAST,0000000000000000000000001111111111111111111111...,2566.0,0,GENDER_MALE,[],1,"[0, 11000]",[],[],[],7.1,CURRENT,[],NONE,NONE,UNLIMITED,"[AGE_BETWEEN_18_23, AGE_BETWEEN_24_30, AGE_BET...",NaN,NaN,NaN,28803,127,16570.17,772,372.0,5.0,0.022450,3314.034000,1,1,0.006477


In [65]:
plan['ac'].value_counts()

[WIFI, 4G]    346
[]            225
[4G, WIFI]    165
Name: ac, dtype: int64

In [87]:
plan.shape

(736, 29)

In [100]:
np.setdiff1d(df.columns,plan.columns)

array(['amount', 'amount_ok', 'channel_id', 'create_role_num',
       'create_time', 'label_ids', 'new_role_money', 'pay_cost',
       'pay_cost_ok', 'pay_rate', 'pay_role_user_num', 'roi', 'source_id'],
      dtype=object)

In [98]:
df['flow_control_mode'].value_counts()

FLOW_CONTROL_MODE_FAST       115
FLOW_CONTROL_MODE_BALANCE     13
FLOW_CONTROL_MODE_SMOOTH       1
Name: flow_control_mode, dtype: int64

In [102]:
import requests

In [103]:
requests.__version__

'2.24.0'

In [104]:
len(plan.columns)

33

In [101]:
plan.columns

Index(['ad_account_id', 'image_id', 'game_id', 'platform', 'android_osv',
       'ios_osv', 'delivery_range', 'inventory_type', 'city', 'location_type',
       'gender', 'age', 'retargeting_type', 'retargeting_tags_include',
       'retargeting_tags_exclude', 'interest_action_mode', 'action_scene',
       'action_days', 'action_categories', 'interest_categories', 'ac',
       'launch_price', 'auto_extend_enabled', 'hide_if_exists',
       'smart_bid_type', 'smart_bid_type', 'adjust_cpa', 'cpa_bid', 'bid2',
       'schedule_time', 'budget', 'hide_if_converted', 'flow_control_mode'],
      dtype='object')

In [97]:
plan.head()

,ad_account_id,image_id,game_id,platform,android_osv,ios_osv,delivery_range,inventory_type,city,location_type,gender,age,retargeting_type,retargeting_tags_include,retargeting_tags_exclude,interest_action_mode,action_scene,action_days,action_categories,interest_categories,ac,launch_price,auto_extend_enabled,hide_if_exists,smart_bid_type,smart_bid_type,adjust_cpa,cpa_bid,bid2,schedule_time,budget,hide_if_converted
0,6620,25547,1001379,[ANDROID],8.0,NONE,DEFAULT,"[INVENTORY_AWEME_FEED, INVENTORY_FEED]",[],CURRENT,NONE,"[AGE_BETWEEN_24_30, AGE_BETWEEN_31_40, AGE_BET...",RETARGETING_EXCLUDE,[],"[315271462, 315271363]",UNLIMITED,NaN,NaN,NaN,[],[],"[0, 11000]",0,0,SMART_BID_CUSTOM,SMART_BID_CUSTOM,0,2896.46,2896.46,0000000000000000000000000011111111111111111111...,3300,CUSTOMER
1,6620,25547,1001379,[ANDROID],8.0,NONE,DEFAULT,"[INVENTORY_HOTSOON_FEED, INVENTORY_UNION_SLOT,...",[],CURRENT,GENDER_MALE,"[AGE_BETWEEN_18_23, AGE_BETWEEN_24_30, AGE_BET...",NONE,[],[],UNLIMITED,NaN,NaN,NaN,[],[],"[0, 11000]",0,0,SMART_BID_CUSTOM,SMART_BID_CUSTOM,0,2988.00,2988.00,0000000000000000000000000011111111111111111111...,3300,CUSTOMER
2,6620,25547,1001379,[ANDROID],8.0,NONE,DEFAULT,"[INVENTORY_FEED, INVENTORY_AWEME_FEED, INVENTO...",[],ALL,NONE,"[AGE_BETWEEN_24_30, AGE_BETWEEN_31_40, AGE_BET...",RETARGETING_EXCLUDE,[],"[307112933, 307112706, 307112436, 306430191, 3...",UNLIMITED,NaN,NaN,NaN,[],[],"[0, 11000]",0,0,SMART_BID_CUSTOM,SMART_BID_CUSTOM,0,2888.00,2888.00,0000000000000000000000000000111111111111111111...,3300,CUSTOMER
3,6620,25547,1001379,[ANDROID],8.0,NONE,DEFAULT,"[INVENTORY_AWEME_FEED, INVENTORY_FEED]","[140100, 340100, 11, 12, 440600, 350100, 37020...",CURRENT,GENDER_MALE,"[AGE_BETWEEN_18_23, AGE_BETWEEN_24_30, AGE_BET...",RETARGETING_INCLUDE,"[307643350, 307609660, 301346170, 307643278, 3...",[308406498],UNLIMITED,NaN,NaN,NaN,[],"[4G, WIFI]","[0, 11000]",0,0,SMART_BID_CUSTOM,SMART_BID_CUSTOM,0,2888.00,2888.00,0000000000000000000000111111111111111111111111...,3300,CUSTOMER
4,6620,25547,1001379,[ANDROID],8.0,NONE,DEFAULT,"[INVENTORY_FEED, INVENTORY_HOTSOON_FEED, INVEN...","[64, 65, 11, 12, 13, 14, 15, 21, 22, 23, 31, 3...",CURRENT,GENDER_MALE,"[AGE_BETWEEN_24_30, AGE_BETWEEN_31_40, AGE_BET...",RETARGETING_EXCLUDE,[],[316309615],UNLIMITED,NaN,NaN,NaN,[],"[WIFI, 4G]","[0, 11000]",0,1,SMART_BID_CUSTOM,SMART_BID_CUSTOM,0,2888.00,2888.00,0000000000000000000000111111111111111111111111...,3300,CUSTOMER


In [313]:
bb

,retargeting_type,retargeting_tags_include,retargeting_tags_exclude
697,RETARGETING_EXCLUDE,[],"[315271462, 315271363]"
339,NONE,[],[]
1120,RETARGETING_EXCLUDE,[],"[313870521, 313252826, 311668665, 311668520, 3..."
3,NONE,[],[]
1132,RETARGETING_INCLUDE,"[314333115, 313693507, 313619617, 313179975, 3...","[313870521, 311595590]"
...,...,...,...
1373,NONE,[],[]
178,RETARGETING_EXCLUDE,[],"[315271462, 315271363]"
654,RETARGETING_EXCLUDE,[],[313870521]
680,RETARGETING_INCLUDE,"[316250745, 316240150]",[]


In [266]:
plan.shape

(736, 7)

In [132]:
df[df['image_id']==28603]

,ad_account_id,game_id,channel_id,source_id,bid2,create_time,smart_bid_type,hide_if_exists,budget,budget,delivery_range,adjust_cpa,inventory_type,inventory_type,hide_if_converted,flow_control_mode,schedule_time,cpa_bid,auto_extend_enabled,gender,city,platform,launch_price,retargeting_tags_exclude,interest_categories,interest_words,ac,android_osv,location_type,retargeting_tags_include,retargeting_type,ios_osv,interest_action_mode,age,action_categories,action_days,action_scene,action_words,image_id,label_ids,amount,create_role_num,new_role_money,pay_role_user_num,roi,pay_cost,pay_cost_ok,amount_ok
935,5924,1001379,21208,452945,2888.0,2021-1-28 14:16:20,SMART_BID_CUSTOM,0,3288,3288.0,DEFAULT,0,"INVENTORY_AWEME_FEED,",[INVENTORY_AWEME_FEED],ADVERTISER,FLOW_CONTROL_MODE_FAST,0000000000000000000000000000111111111111111111...,2888.0,0,GENDER_MALE,[],1,"[0, 11000]","[311886780, 311886984, 311914394]",[],[],"[WIFI, 4G]",8.0,CURRENT,[],RETARGETING_EXCLUDE,NONE,UNLIMITED,"['AGE_BETWEEN_18_23', 'AGE_BETWEEN_24_30', 'AG...",NaN,NaN,NaN,NaN,28603.0,"-1,127",10603.92,175,242.0,3.0,0.022822,3534.64,1,1


In [115]:
df['game_id'].unique()

array([1001379, 1001400], dtype=int64)

In [116]:
df['image_id'].unique()

array([28803., 29403., 25547., 27672., 26241., 26441., 25825., 28976.,
       29075., 29160., 28722., 29356., 28609., 29226., 28678., 28738.,
          nan, 27742., 25562., 27434., 29342., 29347., 29333., 28603.,
       29363., 29045., 28891., 29000., 29437., 29384., 28541., 28800.])

In [129]:
np.intersect1d(df['image_id'].unique(),scroe_image)

array([25547., 25562., 25825., 26241., 26441., 27672., 28541., 28609.,
       28678., 28722., 28738., 28800., 28803., 28891., 28976., 29000.,
       29045., 29075., 29160., 29226., 29333., 29356., 29437.])

In [131]:
np.setdiff1d(df['image_id'].unique(),scroe_image)

array([27434., 27742., 28603., 29342., 29347., 29363., 29384., 29403.,
          nan])

In [ ]:
np.setdiff1d(df['image_id'].unique(),df['image_id'].unique())

In [95]:
df['bid2'].min()

496.54

In [96]:
df['bid2'].max()

3000.0

In [97]:
df['bid2'].mean()

2839.5567424242417

In [118]:
df['age']= df['age'].astype(str)

In [120]:
df['age'].value_counts()

['AGE_BETWEEN_24_30', 'AGE_BETWEEN_31_40', 'AGE_BETWEEN_41_49']                                         35
[]                                                                                                      25
['AGE_ABOVE_50', 'AGE_BETWEEN_18_23', 'AGE_BETWEEN_24_30', 'AGE_BETWEEN_31_40', 'AGE_BETWEEN_41_49']    19
['AGE_BETWEEN_18_23', 'AGE_BETWEEN_24_30', 'AGE_BETWEEN_31_40', 'AGE_BETWEEN_41_49']                    15
['AGE_BETWEEN_18_23', 'AGE_BETWEEN_24_30', 'AGE_BETWEEN_31_40']                                         13
['AGE_BETWEEN_24_30', 'AGE_BETWEEN_31_40']                                                              13
['AGE_BETWEEN_18_23', 'AGE_BETWEEN_24_30', 'AGE_BETWEEN_31_40', 'AGE_BETWEEN_41_49', 'AGE_ABOVE_50']    12
Name: age, dtype: int64

In [98]:
df['smart_bid_type'].unique()

array(['SMART_BID_CONSERVATIVE', 'SMART_BID_CUSTOM'], dtype=object)

In [121]:
df['action_categories'].values

array([nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       list([16]), nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, list([]), nan,
       list([]), nan, nan, list([16, 2, 4]), nan, list([17]),
       list([160407, 160401, 160413, 160419, 160426, 160437, 160443]),
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, list([16, 18, 17, 20]),
       list([16, 18, 17, 20]), nan, nan, list([16, 18, 17, 20]),
       list([16, 18, 17, 20]), nan, nan, nan, nan, nan, nan, nan,
       list([16]), nan, list([16]), nan, list([16]), nan, nan, nan, nan,
       nan, list([16]), nan, nan, nan, nan, nan, nan], dtype=object)

In [130]:
df['inventory_type'].values

array([list(['INVENTORY_AWEME_FEED', 'INVENTORY_FEED']),
       list(['INVENTORY_AWEME_FEED', 'INVENTORY_FEED']),
       list(['INVENTORY_AWEME_FEED', 'INVENTORY_FEED']),
       list(['INVENTORY_AWEME_FEED', 'INVENTORY_FEED']),
       list(['INVENTORY_AWEME_FEED', 'INVENTORY_FEED']),
       list(['INVENTORY_AWEME_FEED', 'INVENTORY_FEED']),
       list(['INVENTORY_AWEME_FEED', 'INVENTORY_FEED']),
       list(['INVENTORY_AWEME_FEED', 'INVENTORY_FEED']),
       list(['INVENTORY_AWEME_FEED', 'INVENTORY_FEED']),
       list(['INVENTORY_AWEME_FEED', 'INVENTORY_FEED']),
       list(['INVENTORY_AWEME_FEED', 'INVENTORY_FEED']),
       list(['INVENTORY_AWEME_FEED', 'INVENTORY_FEED']),
       list(['INVENTORY_AWEME_FEED', 'INVENTORY_FEED']),
       list(['INVENTORY_AWEME_FEED', 'INVENTORY_FEED']),
       list(['INVENTORY_AWEME_FEED', 'INVENTORY_FEED']),
       list(['INVENTORY_AWEME_FEED', 'INVENTORY_FEED']),
       list(['INVENTORY_AWEME_FEED', 'INVENTORY_FEED']),
       list(['INVENTORY_AWEME_F

In [106]:
df.shape

(132, 46)

In [ ]:
df['']

In [105]:
df.isna().sum()

ad_account_id                 0
game_id                       0
channel_id                    0
source_id                     0
bid2                          0
create_time                   0
smart_bid_type                0
hide_if_exists                0
budget                        0
budget                        0
delivery_range                0
adjust_cpa                    0
inventory_type                0
inventory_type                0
hide_if_converted             0
flow_control_mode             0
schedule_time                 0
cpa_bid                       0
auto_extend_enabled           0
gender                        0
city                          0
platform                      0
launch_price                  0
retargeting_tags_exclude      0
interest_categories           0
interest_words                0
ac                            0
android_osv                   0
location_type                 0
retargeting_tags_include      0
retargeting_type              0
ios_osv 